In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter= ',')

unscaled_inputs_all = raw_csv_data[:1:-1]
targets_all = raw_csv_data[:,-1]

In [3]:
raw_csv_data = np.loadtxt('C:\\Users\\Seba\\Documents\\python\\The Data Science Course 2019 - All Resources\\Part_7_Deep_Learning\\S51_L356\\Audiobooks_data.csv', delimiter= ',')

In [4]:
#balance dataset

num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove= []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)



C:\Users\Seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  del sys.path[0]


In [5]:
#standarize the inputs

scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)


In [6]:
#shuffle the data

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]


In [7]:
#split the dataset into train,validation, and test

samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets/train_samples_count))
print(np.sum(validation_targets), validation_samples_count ,np.sum(validation_targets/validation_samples_count))
print(np.sum(test_targets), test_samples_count, np.sum(test_targets/test_samples_count))

1791.0 3579 0.5004191114836547
232.0 447 0.5190156599552571
214.0 448 0.4776785714285714


In [8]:
np.random.shuffle(shuffled_indices)

In [9]:
np.savez('Audiobooks_data_train',inputs = train_inputs, targets= train_targets)
np.savez('Audiobooks_data_validation',inputs = validation_inputs, targets= validation_targets)
np.savez('Audiobooks_data_test',inputs = test_inputs, targets= test_targets)

# Machine Learning Algorithm

In [10]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
valiation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [23]:
#MODEL

input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs= max_epochs,
          callbacks=[early_stopping],
          validation_data=(valiation_inputs,validation_targets),
          verbose=2)


Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.6451 - acc: 0.6681 - val_loss: 0.6379 - val_acc: 0.6868
Epoch 2/100
3579/3579 - 0s - loss: 0.6085 - acc: 0.6887 - val_loss: 0.6233 - val_acc: 0.6890
Epoch 3/100
3579/3579 - 0s - loss: 0.6036 - acc: 0.6924 - val_loss: 0.6179 - val_acc: 0.6779
Epoch 4/100
3579/3579 - 0s - loss: 0.5971 - acc: 0.6957 - val_loss: 0.6167 - val_acc: 0.6823
Epoch 5/100
3579/3579 - 0s - loss: 0.5930 - acc: 0.6999 - val_loss: 0.6213 - val_acc: 0.6868
Epoch 6/100
3579/3579 - 0s - loss: 0.5893 - acc: 0.7019 - val_loss: 0.6129 - val_acc: 0.6779
Epoch 7/100
3579/3579 - 0s - loss: 0.5848 - acc: 0.7035 - val_loss: 0.6107 - val_acc: 0.6779
Epoch 8/100
3579/3579 - 0s - loss: 0.5843 - acc: 0.7086 - val_loss: 0.6221 - val_acc: 0.6801
Epoch 9/100
3579/3579 - 0s - loss: 0.5820 - acc: 0.7072 - val_loss: 0.6148 - val_acc: 0.6823


In [24]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 20us/sample - loss: 0.6659 - acc: 0.6652


In [16]:
print('\n Test loss: {1:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


 Test loss: 66.29. Test accuracy: 66.29%
